In [135]:
import pandas as pd
import requests
import json
from config import gkey

In [136]:
#init dataframe
data = pd.read_csv("Resources\County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
#data.head()
data[["RegionID", "RegionType", "RegionName"]].head()

,RegionID,RegionType,RegionName
0,3101,County,Los Angeles County
1,139,County,Cook County
2,1090,County,Harris County
3,2402,County,Maricopa County
4,2841,County,San Diego County


In [137]:
#api call
coord_df = data.apply(
    lambda row: requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={row.RegionName}&key={gkey}").json()["results"][0]["geometry"]["location"], axis =1, result_type="expand")

In [138]:
#add regionid column & rename lat&lng
coord_df["RegionID"] = data["RegionID"]
coord_df = coord_df.rename({"lat":"Latitude", "lng":"Longitude"},axis=1)
coord_df.head()

,Latitude,Longitude,RegionID
0,34.052227,-118.243660,3101
1,41.737659,-87.697554,139
2,29.775182,-95.310250,1090
3,33.291797,-112.429146,2402
4,32.715730,-117.161097,2841


In [139]:
#join dataframes by Region ID
data_coords = data[["RegionID", "RegionType", "RegionName"]].join(coord_df.set_index("RegionID"), on="RegionID")
data_coords

,RegionID,RegionType,RegionName,Latitude,Longitude
0,3101,County,Los Angeles County,34.052227,-118.243660
1,139,County,Cook County,41.737659,-87.697554
2,1090,County,Harris County,29.775182,-95.310250
3,2402,County,Maricopa County,33.291797,-112.429146
4,2841,County,San Diego County,32.715730,-117.161097
...,...,...,...,...,...
2821,425,County,Camas County,43.509369,-114.824366
2822,1648,County,Daggett County,40.905384,-109.521106
2823,2345,County,Hinsdale County,37.931631,-107.384883
2824,1933,County,Mineral County,39.406899,-78.883555


In [141]:
data_coords.to_csv("Resources/Coords_County_zhvi.csv",index=False)